# Laboratorio 2: De Jupyter a Producción
## Detección de Landmarks Faciales con Streamlit

**Materia**: Procesamiento Digital de Imágenes e Introducción a Visión por Computadora  
**Objetivo**: Convertir un notebook de análisis de landmarks faciales en una aplicación web interactiva desplegada en Streamlit Community Cloud

---

## Contexto Pedagógico

En el **Laboratorio 1** trabajamos con:
- Hugging Face Spaces
- Modelos pre-entrenados
- Entornos virtuales
- Gradio para interfaces
- Kilo como agente de desarrollo

En este **Laboratorio 2** vamos a dar un paso más profesional:
- Usar **VS Code** como IDE principal
- Trabajar con **Kilo + Code Supernova 1M** como agente AI ([https://kilocode.ai/](https://kilocode.ai/))
- Convertir notebooks en scripts Python modulares
- Crear interfaces con **Streamlit** ([https://streamlit.io/](https://streamlit.io/))
- Desplegar en **Streamlit Community Cloud**

---

## Parte 1: Fundamentos Teóricos

### ¿Qué son los Landmarks Faciales?

Los **Landmarks Faciales** son puntos de referencia específicos que mapean las características clave de un rostro humano. Pensá en ellos como un "esqueleto" invisible que describe la geometría facial.

![MediaPipe Face Mesh](https://ai.google.dev/static/mediapipe/images/solutions/face_landmarker_keypoints.png?hl=es-419)

**Referencia oficial**: [MediaPipe Face Landmarker](https://ai.google.dev/edge/mediapipe/solutions/vision/face_landmarker)

**Cantidad de puntos**: MediaPipe Face Mesh detecta **478 landmarks** que cubren:
- Contorno facial (mandíbula, frente)
- Ojos (iris, párpados, cejas)
- Nariz (puente, fosas nasales)
- Boca (labios, comisuras)

### Aplicaciones en la Industria

| Aplicación | Descripción | Ejemplo Real |
|------------|-------------|-------------|
| **Filtros AR** | Superponer objetos virtuales en tiempo real | Instagram, Snapchat, TikTok |
| **Análisis de Expresiones** | Detectar emociones para UX testing | Microsoft Azure Emotion API |
| **Animación Facial** | Captura de movimiento para CGI | Avatar, The Mandalorian |
| **Autenticación** | Face ID, reconocimiento facial | Apple Face ID, bancos |
| **Salud** | Detección de parálisis facial, análisis genético | Investigación médica |
| **Gaming** | Control de personajes con expresiones | VR/AR gaming |

### Stack Tecnológico

```
MediaPipe (Google) → Modelo pre-entrenado de detección
    ↓
OpenCV → Procesamiento de imágenes
    ↓
Streamlit → Interfaz web interactiva
    ↓
Streamlit Cloud → Hosting y deployment
```

---

## Parte 2: Arquitectura del Proyecto

### Estructura de Directorios (objetivo final)

```
facial-landmarks-app/
│
├── venv/                    # Entorno virtual (no subir a Git)
│
├── src/
│   ├── detector.py          # Lógica de detección de landmarks
│   ├── utils.py             # Funciones auxiliares (conversión de imágenes)
│   └── config.py            # Configuración (parámetros del modelo)
│
├── app.py                   # Aplicación Streamlit principal
│
├── requirements.txt         # Dependencias del proyecto
├── .gitignore              # Archivos a ignorar en Git
└── README.md               # Documentación del proyecto
```

### Flujo de Trabajo

El proceso de desarrollo sigue estas etapas:

```
1. Jupyter Notebook
   ↓
2. Análisis y Prototipado
   ↓
3. Extracción de Lógica
   ↓
4. Scripts Python Modulares
   ↓
5. Interfaz Streamlit
   ↓
6. Testing Local
   ↓
7. Deployment en Streamlit Cloud
```

---

## Parte 3: Paso a Paso - Desarrollo del MVP

### Fase 1: Configuración del Entorno de Desarrollo

#### Paso 1.1: Instalación de VS Code y Extensiones

**Descargar VS Code**: [https://code.visualstudio.com/](https://code.visualstudio.com/)

**Extensiones necesarias**:
```bash
# Abrir VS Code y buscar en Extensions (Ctrl+Shift+X):
1. Python (Microsoft)
2. Jupyter (Microsoft)
3. Pylance (Microsoft)
4. GitLens (opcional pero recomendado)
```

#### Paso 1.2: Configuración de Kilo con Code Supernova

**Kilo** es un agente AI integrado en VS Code. Para este lab, vamos a usarlo con el modelo **Code Supernova 1M**.

```bash
# En la terminal de VS Code:
# (Los pasos exactos dependen de cómo tengan configurado Kilo en su institución)

# Ejemplo conceptual:
kilo config --model code-supernova-1m
kilo auth login
```

**Tip**: Usá Kilo para:
- Autocompletar código
- Explicar funciones de MediaPipe
- Refactorizar código del notebook
- Generar docstrings

---

### Fase 2: Creación del Entorno Virtual

#### Paso 2.1: Crear el Proyecto

```bash
# Crear directorio del proyecto
mkdir facial-landmarks-app
cd facial-landmarks-app

# Crear entorno virtual
python -m venv venv

# Activar el entorno virtual
# En Windows:
venv\Scripts\activate

# En Linux/Mac:
source venv/bin/activate
```

**Verificar activación**: Deberías ver `(venv)` antes del prompt en la terminal.

#### Paso 2.2: Instalar Dependencias Base

```bash
# Instalar las bibliotecas necesarias
pip install mediapipe opencv-python streamlit pillow numpy

# Generar requirements.txt
pip freeze > requirements.txt
```

**Contenido de `requirements.txt` esperado**:
```
mediapipe==0.10.21
opencv-python==4.11.0.86
streamlit==1.39.0
Pillow==11.2.1
numpy==1.26.4
```

---

### Fase 3: Conversión de Notebook a Scripts Python

#### Paso 3.1: Analizar el Código del Notebook Original

Del notebook `Landmarks_Faciales.ipynb`, identificamos estos componentes clave:

```python
# 1. INICIALIZACIÓN DEL MODELO
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

# 2. PROCESAMIENTO DE IMAGEN
imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
resultados = mp_face_mesh.process(imagen_rgb)

# 3. DIBUJO DE LANDMARKS
for punto in rostro.landmark:
    coord_x_pixel = int(punto.x * ancho)
    coord_y_pixel = int(punto.y * alto)
    cv2.circle(imagen_con_puntos, (coord_x_pixel, coord_y_pixel), 2, (0, 255, 0), -1)
```

#### Paso 3.2: Crear `src/config.py`

```python
# src/config.py
"""
Configuración del detector de landmarks faciales.
"""

# Parámetros del modelo MediaPipe
FACE_MESH_CONFIG = {
    "static_image_mode": True,
    "max_num_faces": 1,
    "refine_landmarks": True,
    "min_detection_confidence": 0.5
}

# Configuración de visualización
LANDMARK_COLOR = (0, 255, 0)  # Verde en BGR
LANDMARK_RADIUS = 2
LANDMARK_THICKNESS = -1  # Relleno

# Cantidad de landmarks esperados
TOTAL_LANDMARKS = 478
```

#### Paso 3.3: Crear `src/utils.py`

```python
# src/utils.py
"""
Funciones auxiliares para procesamiento de imágenes.
"""
import cv2
import numpy as np
from PIL import Image


def pil_to_cv2(pil_image):
    """
    Convierte una imagen PIL a formato OpenCV (numpy array BGR).
    
    Args:
        pil_image (PIL.Image): Imagen en formato PIL
    
    Returns:
        numpy.ndarray: Imagen en formato OpenCV (BGR)
    """
    # Convertir PIL a RGB numpy array
    rgb_array = np.array(pil_image.convert('RGB'))
    # Convertir RGB a BGR (formato OpenCV)
    bgr_array = cv2.cvtColor(rgb_array, cv2.COLOR_RGB2BGR)
    return bgr_array


def cv2_to_pil(cv2_image):
    """
    Convierte una imagen OpenCV a formato PIL.
    
    Args:
        cv2_image (numpy.ndarray): Imagen en formato OpenCV (BGR)
    
    Returns:
        PIL.Image: Imagen en formato PIL (RGB)
    """
    # Convertir BGR a RGB
    rgb_array = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
    # Convertir a PIL
    pil_image = Image.fromarray(rgb_array)
    return pil_image


def resize_image(image, max_width=800):
    """
    Redimensiona la imagen manteniendo el aspect ratio.
    
    Args:
        image (numpy.ndarray): Imagen OpenCV
        max_width (int): Ancho máximo deseado
    
    Returns:
        numpy.ndarray: Imagen redimensionada
    """
    alto, ancho = image.shape[:2]
    
    if ancho > max_width:
        ratio = max_width / ancho
        nuevo_ancho = max_width
        nuevo_alto = int(alto * ratio)
        image = cv2.resize(image, (nuevo_ancho, nuevo_alto))
    
    return image
```

#### Paso 3.4: Crear `src/detector.py`

```python
# src/detector.py
"""
Detector de landmarks faciales usando MediaPipe.
"""
import cv2
import mediapipe as mp
from .config import FACE_MESH_CONFIG, LANDMARK_COLOR, LANDMARK_RADIUS, LANDMARK_THICKNESS


class FaceLandmarkDetector:
    """
    Clase para detectar y visualizar landmarks faciales.
    """
    
    def __init__(self):
        """Inicializa el detector de MediaPipe."""
        self.face_mesh = mp.solutions.face_mesh.FaceMesh(**FACE_MESH_CONFIG)
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
    
    def detect(self, image):
        """
        Detecta landmarks faciales en la imagen.
        
        Args:
            image (numpy.ndarray): Imagen en formato BGR (OpenCV)
        
        Returns:
            tuple: (imagen_procesada, landmarks, info)
                - imagen_procesada: imagen con landmarks dibujados
                - landmarks: objeto de landmarks de MediaPipe
                - info: diccionario con información de detección
        """
        # Convertir BGR a RGB para MediaPipe
        imagen_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen
        resultados = self.face_mesh.process(imagen_rgb)
        
        # Crear copia para dibujar
        imagen_con_puntos = image.copy()
        
        info = {
            "rostros_detectados": 0,
            "total_landmarks": 0,
            "deteccion_exitosa": False
        }
        
        # Si se detectaron rostros
        if resultados.multi_face_landmarks:
            info["rostros_detectados"] = len(resultados.multi_face_landmarks)
            
            # Tomar el primer rostro
            rostro = resultados.multi_face_landmarks[0]
            info["total_landmarks"] = len(rostro.landmark)
            info["deteccion_exitosa"] = True
            
            # Dibujar landmarks
            alto, ancho = image.shape[:2]
            
            for punto in rostro.landmark:
                coord_x_pixel = int(punto.x * ancho)
                coord_y_pixel = int(punto.y * alto)
                
                cv2.circle(
                    imagen_con_puntos,
                    (coord_x_pixel, coord_y_pixel),
                    LANDMARK_RADIUS,
                    LANDMARK_COLOR,
                    LANDMARK_THICKNESS
                )
            
            return imagen_con_puntos, rostro, info
        
        # No se detectó rostro
        return imagen_con_puntos, None, info
    
    def close(self):
        """Libera recursos del detector."""
        self.face_mesh.close()
```

---

### Fase 4: Desarrollo de la Interfaz Streamlit

#### Paso 4.1: Crear `app.py`

```python
# app.py
"""
Aplicación Streamlit para detección de landmarks faciales.
"""
import streamlit as st
from PIL import Image
from src.detector import FaceLandmarkDetector
from src.utils import pil_to_cv2, cv2_to_pil, resize_image
from src.config import TOTAL_LANDMARKS


# Configuración de la página
st.set_page_config(
    page_title="Detector de Landmarks Faciales",
    layout="wide"
)

# Título y descripción
st.title("Detector de Landmarks Faciales")
st.markdown("""
Esta aplicación detecta **478 puntos clave** en rostros humanos usando MediaPipe.
Subí una imagen con un rostro y mirá la magia de la visión por computadora.
""")

# Sidebar con información
with st.sidebar:
    st.header("Información")
    st.markdown("""
    ### ¿Qué son los Landmarks?
    Son puntos de referencia que mapean:
    - Ojos (iris, párpados)
    - Nariz (puente, fosas)
    - Boca (labios, comisuras)
    - Contorno facial
    
    ### Aplicaciones
    - Filtros AR (Instagram)
    - Análisis de expresiones
    - Animación facial
    - Autenticación biométrica
    """)
    
    st.divider()
    st.caption("Desarrollado en el Laboratorio 2 - IFTS24")

# Uploader de imagen
uploaded_file = st.file_uploader(
    "Subí una imagen con un rostro",
    type=["jpg", "jpeg", "png"],
    help="Formatos aceptados: JPG, JPEG, PNG"
)

if uploaded_file is not None:
    # Cargar imagen
    imagen_original = Image.open(uploaded_file)
    
    # Convertir a formato OpenCV
    imagen_cv2 = pil_to_cv2(imagen_original)
    
    # Redimensionar si es muy grande
    imagen_cv2 = resize_image(imagen_cv2, max_width=800)
    
    # Columnas para mostrar antes/después
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Imagen Original")
        st.image(cv2_to_pil(imagen_cv2), use_container_width=True)
    
    # Detectar landmarks
    with st.spinner("Detectando landmarks faciales..."):
        detector = FaceLandmarkDetector()
        imagen_procesada, landmarks, info = detector.detect(imagen_cv2)
        detector.close()
    
    with col2:
        st.subheader("Landmarks Detectados")
        st.image(cv2_to_pil(imagen_procesada), use_container_width=True)
    
    # Mostrar información de detección
    st.divider()
    
    if info["deteccion_exitosa"]:
        st.success("Detección exitosa")
        
        # Métricas
        metric_col1, metric_col2, metric_col3 = st.columns(3)
        
        with metric_col1:
            st.metric("Rostros detectados", info["rostros_detectados"])
        
        with metric_col2:
            st.metric("Landmarks detectados", f"{info['total_landmarks']}/{TOTAL_LANDMARKS}")
        
        with metric_col3:
            porcentaje = (info['total_landmarks'] / TOTAL_LANDMARKS) * 100
            st.metric("Precisión", f"{porcentaje:.1f}%")
    else:
        st.error("No se detectó ningún rostro en la imagen")
        st.info("""
        **Consejos**:
        - Asegurate de que el rostro esté bien iluminado
        - El rostro debe estar mirando hacia la cámara
        - Probá con una imagen de mayor calidad
        """)

else:
    # Mensaje de bienvenida
    st.info("Subí una imagen para comenzar la detección")
    
    # Ejemplo visual
    st.markdown("### Ejemplo de Resultado")
    st.image(
        "https://ai.google.dev/static/mediapipe/images/solutions/face_landmarker_keypoints.png?hl=es-419",
        caption="MediaPipe detecta 478 landmarks faciales",
        width=400
    )
```

#### Paso 4.2: Probar la Aplicación Localmente

```bash
# En la terminal con el venv activado:
streamlit run app.py
```

Esto debería abrir automáticamente `http://localhost:8501` en tu navegador.

**Checklist de Testing**:
- [ ] La aplicación carga sin errores
- [ ] Podés subir una imagen
- [ ] Se detectan los landmarks correctamente
- [ ] Las métricas se muestran bien
- [ ] El layout es responsive

---

### Fase 5: Preparación para Deployment

#### Paso 5.1: Crear `.gitignore`

```gitignore
# .gitignore

# Entorno virtual
venv/
env/
.venv/

# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python

# Jupyter
.ipynb_checkpoints

# IDE
.vscode/
.idea/

# Sistema
.DS_Store
Thumbs.db

# Streamlit
.streamlit/secrets.toml
```

#### Paso 5.2: Crear `README.md`

```markdown
# Detector de Landmarks Faciales

Aplicación web para detectar 478 puntos clave en rostros humanos usando MediaPipe y Streamlit.

## Características

- Detección de 478 landmarks faciales
- Interfaz web interactiva
- Procesamiento en tiempo real
- Visualización antes/después

## Tecnologías

- **MediaPipe**: Detección de landmarks
- **OpenCV**: Procesamiento de imágenes
- **Streamlit**: Framework web
- **Python 3.11+**

## Instalación Local

```bash
# Clonar repositorio
git clone https://github.com/tu-usuario/facial-landmarks-app.git
cd facial-landmarks-app

# Crear entorno virtual
python -m venv venv
venv\Scripts\activate  # Windows
source venv/bin/activate  # Linux/Mac

# Instalar dependencias
pip install -r requirements.txt

# Ejecutar aplicación
streamlit run app.py
```

## Deployment en Streamlit Community Cloud

1. Subir el código a GitHub
2. Ir a [https://share.streamlit.io](https://share.streamlit.io)
3. Conectar tu repositorio
4. Configurar el archivo principal como `app.py`
5. Deploy

## Documentación

- [MediaPipe Face Landmarker](https://ai.google.dev/edge/mediapipe/solutions/vision/face_landmarker)
- [Streamlit Docs](https://docs.streamlit.io)
- [Kilo Code](https://kilocode.ai/)

## Autor

Desarrollado como parte del Laboratorio 2 - IFTS24  
Materia: Procesamiento Digital de Imágenes

## Licencia

MIT License
```

---

### Fase 6: Deployment en Streamlit Community Cloud

#### Paso 6.1: Preparar el Repositorio Git

```bash
# Inicializar repositorio
git init
git add .
git commit -m "Initial commit: Facial landmarks detector"

# Crear repositorio en GitHub (desde la web)
# Luego conectar:
git remote add origin https://github.com/tu-usuario/facial-landmarks-app.git
git branch -M main
git push -u origin main
```

#### Paso 6.2: Deployment en Streamlit Community Cloud

**Sitio oficial**: [https://streamlit.io/](https://streamlit.io/)

**Pasos para desplegar**:

1. **Ir a** [https://share.streamlit.io](https://share.streamlit.io)
2. **Conectar** tu cuenta de GitHub
3. **Seleccionar** tu repositorio `facial-landmarks-app`
4. **Configurar**:
   - Branch: `main`
   - Main file path: `app.py`
5. **Deploy**

Tu app estará disponible en: `https://tu-usuario-facial-landmarks-app.streamlit.app`

#### Paso 6.3: (Alternativa) Deployment en Hugging Face Spaces

Si preferís usar Hugging Face (del Lab 1):

1. Crear un nuevo Space en [https://huggingface.co/spaces](https://huggingface.co/spaces)
2. Elegir SDK: **Streamlit**
3. Subir los archivos:
   - `app.py`
   - `requirements.txt`
   - Carpeta `src/`

---

## Parte 4: Actividades de Aprendizaje

**Fecha de entrega**: 29 de octubre de 2025

---

### Actividad 1: Implementación Base (Obligatoria)

**Objetivo**: Crear la aplicación funcional siguiendo la guía.

**Entregables**:
1. Repositorio GitHub con el código
2. Aplicación desplegada y funcionando
3. Screenshot de la aplicación detectando landmarks en tu propia foto

**Criterios de evaluación**:
- Estructura de directorios correcta
- Código modularizado en `src/`
- Aplicación funcionando localmente
- Deployment exitoso
- README.md completo

---

### Actividad 2: Mejoras y Extensiones (Optativas)

Elegí **al menos 2** de estas mejoras:

#### Opción A: Múltiples Estilos de Visualización

Agregá opciones para visualizar los landmarks de diferentes formas:
- Puntos + malla conectada (usando `mp.solutions.drawing_utils`)
- Solo contornos principales (ojos, boca, rostro)
- Heatmap de densidad de puntos

**Tip con Kilo**: Pedile que te ayude a usar `mp_drawing.draw_landmarks()`

```python
# Ejemplo para empezar:
mp_drawing.draw_landmarks(
    image=imagen_con_puntos,
    landmark_list=rostro,
    connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
    landmark_drawing_spec=None,
    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
)
```

#### Opción B: Análisis de Expresiones

Calculá métricas simples basadas en landmarks:
- Apertura de boca (distancia entre labios)
- Apertura de ojos
- Inclinación de cabeza

```python
# Ejemplo para distancia entre labios:
import math

def calcular_apertura_boca(landmarks, alto, ancho):
    # Landmark 13: labio superior
    # Landmark 14: labio inferior
    punto_superior = landmarks.landmark[13]
    punto_inferior = landmarks.landmark[14]
    
    y1 = punto_superior.y * alto
    y2 = punto_inferior.y * alto
    
    distancia = abs(y2 - y1)
    return distancia
```

#### Opción C: Procesamiento de Video

Modificá el código para procesar videos frame por frame:

```python
# Cambiar en config.py:
FACE_MESH_CONFIG = {
    "static_image_mode": False,  # Cambio clave
    "max_num_faces": 1,
    "refine_landmarks": True,
    "min_detection_confidence": 0.5,
    "min_tracking_confidence": 0.5  # Nuevo parámetro
}
```

Investigá `st.camera_input()` para captura en vivo.

#### Opción D: Exportar Landmarks a JSON

Agregá un botón para descargar las coordenadas de los landmarks:

```python
import json

def landmarks_to_dict(landmarks, alto, ancho):
    """Convierte landmarks a diccionario exportable."""
    data = []
    for idx, punto in enumerate(landmarks.landmark):
        data.append({
            "id": idx,
            "x": punto.x * ancho,
            "y": punto.y * alto,
            "z": punto.z
        })
    return data

# En Streamlit:
landmarks_json = json.dumps(landmarks_to_dict(landmarks, alto, ancho), indent=2)
st.download_button(
    label="Descargar Landmarks (JSON)",
    data=landmarks_json,
    file_name="landmarks.json",
    mime="application/json"
)
```

---

### Actividad 3: Documentación Técnica (Obligatoria)

Creá un documento `INFORME.md` que incluya:

1. **Introducción**: Explicá qué son los landmarks y su importancia
2. **Arquitectura**: Diagrama de la estructura del proyecto
3. **Decisiones de diseño**: Por qué elegiste cierta estructura de código
4. **Desafíos**: Problemas que encontraste y cómo los resolviste
5. **Uso de Kilo**: Capturas de cómo usaste el agente AI durante el desarrollo
6. **Conclusiones**: Aprendizajes principales

**Formato**: Markdown con imágenes, código y referencias.

---

## Parte 5: Guía de Uso de Kilo como Agente

**Sitio oficial de Kilo**: [https://kilocode.ai/](https://kilocode.ai/)

### ¿Cómo Aprovechar Kilo en Este Proyecto?

Kilo puede ser tu asistente de desarrollo en cada fase:

#### 1. Conversión de Notebook a Scripts

**Prompt para Kilo**:
```
Tengo este código de Jupyter notebook:
[pegar código de detección]

Necesito convertirlo en una clase Python con:
- Método __init__ para configurar MediaPipe
- Método detect() que reciba una imagen y retorne landmarks
- Docstrings en español
- Type hints
```

#### 2. Debugging de Errores

**Ejemplo de uso**:
```python
# Si te sale este error:
# AttributeError: module 'cv2' has no attribute 'cvtColor'

# Preguntale a Kilo:
# "Tengo este error en OpenCV, ¿qué puede estar pasando?"
# [pegar traceback completo]
```

#### 3. Mejoras de Código

**Prompt para refactoring**:
```
Refactorizá esta función para que:
- Maneje excepciones correctamente
- Valide inputs
- Sea más eficiente

[pegar función]
```

#### 4. Explicación de Conceptos

```
"Explicame cómo funciona la detección de landmarks en MediaPipe.
Necesito entenderlo para documentar mi código."
```

#### 5. Generación de Tests

```
"Generá tests unitarios para la clase FaceLandmarkDetector usando pytest.
Necesito testear:
- Detección exitosa con imagen válida
- Manejo de imagen sin rostro
- Conversión de formatos"
```

### Comandos Útiles de Kilo en VS Code

- **Ctrl + I**: Abrir chat de Kilo
- **Seleccionar código + Ctrl + I**: Preguntar sobre código específico
- **Cmd/Ctrl + K**: Autocompletar código

### Mejores Prácticas con Agentes AI

**SÍ hacer**:
- Dar contexto completo del problema
- Pedir explicaciones de código generado
- Usar para aprender patrones de diseño
- Validar siempre el código generado

**NO hacer**:
- Copiar código sin entenderlo
- Depender 100% del agente
- Aceptar código sin testearlo
- Ignorar warnings de seguridad

---

## Parte 6: Checklist de Entrega Final

### Estructura del Proyecto

```
facial-landmarks-app/
├── venv/ (no subir a Git)
├── src/
│   ├── __init__.py
│   ├── detector.py
│   ├── utils.py
│   └── config.py
├── app.py
├── requirements.txt
├── .gitignore
├── README.md
└── INFORME.md (tu documentación)
```

### Funcionalidades Mínimas

- [ ] Subir imagen desde interfaz
- [ ] Detectar landmarks correctamente
- [ ] Mostrar imagen antes/después
- [ ] Mostrar métricas (cantidad de landmarks, rostros detectados)
- [ ] Manejar casos sin detección de rostro
- [ ] Sidebar con información educativa
- [ ] Aplicación desplegada online

### Documentación Requerida

- [ ] README.md con instrucciones de instalación
- [ ] Docstrings en todas las funciones
- [ ] Comentarios explicativos en código complejo
- [ ] INFORME.md con análisis técnico
- [ ] Screenshots de la aplicación funcionando

### Calidad de Código

- [ ] Código modularizado (no todo en un archivo)
- [ ] Nombres de variables descriptivos en español
- [ ] Manejo de excepciones
- [ ] No hay código duplicado
- [ ] Configuración separada de lógica (config.py)

### Deployment

- [ ] Repositorio Git público
- [ ] Aplicación accesible vía URL
- [ ] README incluye link a app desplegada
- [ ] No hay errores en logs de deployment

---

## Recursos Adicionales

### Documentación Oficial

- [MediaPipe Face Mesh Guide](https://ai.google.dev/edge/mediapipe/solutions/vision/face_landmarker?hl=es-419)
- [Streamlit API Reference](https://docs.streamlit.io/library/api-reference)
- [OpenCV Python Tutorials](https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html)
- [Python venv Documentation](https://docs.python.org/3/library/venv.html)

### Tutoriales Recomendados

- [Streamlit from Scratch](https://www.youtube.com/watch?v=JwSS70SZdyM)
- [MediaPipe Hands-On](https://www.youtube.com/watch?v=wyWmWaXapmI)
- [VS Code for Python](https://code.visualstudio.com/docs/python/python-tutorial)

### Inspiración de Proyectos

- [Face Mesh AR Filters](https://github.com/google/mediapipe/tree/master/mediapipe/examples)
- [Emotion Detection App](https://huggingface.co/spaces/nlpconnect/emotion-detection)
- [Streamlit Gallery](https://streamlit.io/gallery)

---

## Preguntas Frecuentes (FAQ)

### 1. ¿Por qué MediaPipe no detecta mi rostro?

**Posibles causas**:
- Imagen muy oscura o borrosa
- Rostro de perfil (debe estar frontal)
- Resolución muy baja
- Rostro parcialmente cubierto

**Solución**: Ajustá `min_detection_confidence` en `config.py` a un valor más bajo (ej: 0.3).

### 2. ¿Cómo convierto el notebook si tengo código muy complejo?

**Estrategia**:
1. Identificá bloques de funcionalidad (detección, visualización, procesamiento)
2. Convertí cada bloque en una función
3. Agrupá funciones relacionadas en módulos
4. Usá Kilo para refactorizar paso a paso

### 3. ¿Streamlit es gratis?

Sí, **Streamlit Community Cloud** es gratis para:
- Repositorios públicos
- Hasta 1GB de recursos
- Apps ilimitadas

### 4. ¿Puedo usar otra interfaz en lugar de Streamlit?

Sí, pero deberías consultar antes. Alternativas:
- **Gradio**: Más simple, menos personalizable
- **Flask**: Más control, más complejidad
- **FastAPI + HTML**: Para APIs REST

### 5. ¿Qué hago si el deployment falla?

**Checklist de troubleshooting**:
1. Verificá que `requirements.txt` esté completo
2. Asegurate de que no haya rutas absolutas en el código
3. Revisá los logs de deployment
4. Comprobá que el `app.py` esté en la raíz
5. Verificá compatibilidad de versiones de librerías

### 6. ¿Cómo mejoro el rendimiento?

**Optimizaciones**:
```python
# 1. Cachear el modelo
@st.cache_resource
def load_detector():
    return FaceLandmarkDetector()

# 2. Redimensionar imágenes grandes
imagen_cv2 = resize_image(imagen_cv2, max_width=800)

# 3. Usar static_image_mode=True para imágenes
```

---

## Criterios de Evaluación

### Rúbrica de Evaluación (Total: 100 puntos)

| Criterio | Puntos | Descripción |
|----------|--------|-------------|
| **Funcionalidad** | 30 | La aplicación detecta landmarks correctamente y maneja errores |
| **Estructura de Código** | 20 | Código modularizado, limpio y organizado en `src/` |
| **Documentación** | 15 | README, docstrings, comentarios e informe técnico |
| **Deployment** | 15 | Aplicación desplegada y accesible online |
| **Uso de Herramientas** | 10 | Uso efectivo de VS Code, venv, Git y Kilo |
| **Mejoras Opcionales** | 10 | Implementación de al menos 2 extensiones |

### Niveles de Logro

**Excelente (90-100)**:
- Todas las funcionalidades base implementadas
- Al menos 2 mejoras opcionales
- Código profesional y bien documentado
- Deployment exitoso con 0 errores
- Informe técnico detallado con análisis profundo

**Muy Bueno (75-89)**:
- Funcionalidades base completas
- 1 mejora opcional
- Código limpio con documentación básica
- Deployment exitoso
- Informe técnico completo

**Bueno (60-74)**:
- Funcionalidades base implementadas
- Sin mejoras opcionales
- Código funcional pero con áreas de mejora
- Deployment con algunos warnings
- Informe básico

**Aprobado (50-59)**:
- Algunas funcionalidades faltantes
- Sin mejoras opcionales
- Código desorganizado
- Deployment con errores menores
- Documentación incompleta

---

## Conclusión

Este laboratorio te lleva desde un **notebook exploratorio** hasta una **aplicación web en producción**, pasando por todas las etapas del ciclo de desarrollo profesional:

1. **Prototipado** en Jupyter
2. **Modularización** en scripts Python
3. **Interfaz de usuario** con Streamlit
4. **Control de versiones** con Git
5. **Deployment** en la nube

Aprendiste a usar herramientas profesionales:
- VS Code como IDE principal
- Kilo como asistente AI de desarrollo
- Entornos virtuales para gestión de dependencias
- Plataformas cloud para deployment

Y aplicaste conceptos clave de visión por computadora:
- Detección de landmarks faciales
- Procesamiento de imágenes con OpenCV
- Modelos pre-entrenados (MediaPipe)

### Próximos Pasos

Podés extender este proyecto para:
- Detectar expresiones faciales (feliz, triste, sorprendido)
- Aplicar filtros AR en tiempo real
- Analizar movimientos faciales para aplicaciones médicas
- Controlar interfaces con gestos faciales

**Éxitos con el laboratorio**

---

**Fecha de entrega**: 29 de octubre de 2025  
**Consultas**: [Email/Canal de comunicación]
